# Slune quick-start demo: simple regularised linear regression

This notebook walks through the minimal "hello, world" workflow for **slune-lib**:

1. Define and train a simple regularised linear regression model.
2. Log an evaluation metric (mean-squared error) using a `SaverCsv`.
3. Submit a small grid search of the regularisation parameter via `sbatchit`.
4. Query the logged results to find the best parameter value.

Prerequisites 📋  
• A machine with SLURM (`sbatch`) available – you can still run the local parts without it.  
• `pip install slune-lib`.

We keep the modelling code intentionally tiny so the focus stays on slune’s API.

In [1]:
# Create/overwrite `model.py` – tiny training script that slune will run on the cluster
import textwrap, pathlib

model_code = textwrap.dedent('''
import argparse
from slune import get_csv_saver

# ---------- tiny "model" ----------
X_TEST = [6, 7, 8]
Y_TEST = [7, 8, 9]

def predict(x, alpha):
    # Toy linear prediction with a regularisation bias
    return 1 + x + alpha


def mse(y_true, y_pred):
    return sum((yt - yp) ** 2 for yt, yp in zip(y_true, y_pred)) / len(y_true)
# ----------------------------------

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--alpha", type=float, required=True)
    args = parser.parse_args()

    predictions = [predict(x, args.alpha) for x in X_TEST]
    metric = mse(Y_TEST, predictions)

    saver = get_csv_saver(params={"alpha": args.alpha})
    saver.log({"mse": metric})
    saver.save_collated()
''')

pathlib.Path("model.py").write_text(model_code)
print("✅ Wrote/updated model.py ({} bytes)".format(len(model_code)))

ValueError: could not convert string to float: '--f=/Users/jd18380/Library/Jupyter/runtime/kernel-v3ad3d407b1de8db4312ce3a7fdc4f697528c064ae.json'

## 1️⃣ Generate the training script

Run the previous cell to create `model.py` in this folder. The script:

* parses `--alpha=<value>` from the command line,
* evaluates a toy regularised linear regression on a dummy dataset,
* logs the mean-squared error with a default `SaverCsv`.

👉  You can open `model.py` to inspect or customise it before queuing jobs.

In [ ]:
from slune.searchers import SearcherGrid
from slune import sbatchit

# 2️⃣ Launch the grid search on the cluster
searcher = SearcherGrid({'alpha': [0.25, 0.5, 0.75]}, runs=1)

sbatchit(
    script_path="model.py",
    sbatch_path="template.sh",  # adjust SBATCH directives to your cluster
    searcher=searcher,
    cargs={},
    saver=None  # provide a Saver to avoid re-running completed jobs
)


Once the jobs have finished (monitor them with `squeue` or your favourite SLURM UI), you can load the collated CSV and inspect the best configuration:

In [ ]:
from slune import get_csv_saver

# 3️⃣ Retrieve results
saver = get_csv_saver()
params, mse = saver.read(params=[], metric_name="mse", select_by="min")
print("Best hyper-parameters:", params)
print("MSE:", mse)
